# Alphabet Soup Charity

## Preprocessing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
# import io # for Google Colab

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("./Resources/charity_data.csv") # Local Jupyter Notebook
# application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv'])) # Google Colab
application_df.head(3)

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(['EIN','NAME'], axis=1)
df.head(3)

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
app_type_value_counts = df['APPLICATION_TYPE'].value_counts()
app_type_value_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for x in range(len(app_type_value_counts)):
    if app_type_value_counts.values[x] < 500:
        application_types_to_replace.append(app_type_value_counts.index[x])
        
# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_val_counts = df['CLASSIFICATION'].value_counts()
classification_val_counts

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] > 1:
        print(f'{classification_val_counts.index[x]}\t{classification_val_counts.values[x]}')

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] < 1500:
        classifications_to_replace.append(classification_val_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df)
print(df_dummies.columns)
df_dummies

In [ ]:
# Split our preprocessed data into our features and target arrays
y_label = df_dummies['IS_SUCCESSFUL']
X_dummies = df_dummies.drop(['IS_SUCCESSFUL'],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

In [ ]:
X_train.shape

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=2), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# !pip install keras-tuner

In [ ]:
# Import the kerastuner library
import keras_tuner as kt
import os

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    directory=os.path.normpath('C:/untitled_project/09'),
    hyperband_iterations=2)

In [ ]:
y_train.shape

In [ ]:
X_train_scaled.shape

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]

# Export our model to HDF5 file
best_model.save('Sequential008.h5')
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")